<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scanpy == 1.8.2 umap-learn == 0.5.3 leidenalg == 0.8.10 pyro-ppl == 1.8.1 scvi == 0.6.8

ERROR: Invalid requirement: '=='
